# finance_report---Balance_ProfitLoss_cashFlow

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

# sID(stock_id) = string
# year = integer
# season = integer
# table = integer (1 = balance 2.profit & lose 3.cash flow)
def finance_report(sID,year,season,table):
    url = f"https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID={sID}&SYEAR={year}&SSEASON={season}&REPORT_ID=C"
    res = requests.get(url)
    res.encoding = 'big5'
    soup = bs(res.text, "lxml")
    tables = soup.select('table')
    season = f"{year}_Q{season}"
    
    # set table name  
    if table == 1:
        name = "balance"
    elif table == 2:
        name = "profit&lose"
    elif table == 3:
        name = "cashflow"
        
    # check tables of  balance, profit&lose, and cashflow exist
    if len(tables) == 1:
        pass
    elif len(tables) > 1:        
        tbl = []
        tr_list = tables[table].select("tr")
        for tr in tr_list: 
            row = [sID,season]    
            td_list = tr.select("td")
            for i,td in enumerate(td_list):           
                # keep data in first & second column 
                if i < 2:
                    row.append(td.text.strip().replace("\n","").replace("（","(").replace("）",")").replace("－","-").replace(",",""))
            if len(row) == 4 and len(row[3]) > 0:
                tbl.append(row)      
        # build table of finance report
        df = pd.DataFrame(tbl,columns = ["sID","year_season","item","value"])
        print(name)
        print(df)


    

# example sID = 2330 ,year = 2015 ,season = 4
for i in range(1,4):
    finance_report(sID = 2330,year = 2015,season = 4,table = i)

balance
     sID year_season                       item       value
0   2330     2015_Q4                  現金及約當現金合計   562688930
1   2330     2015_Q4      透過損益按公允價值衡量之金融資產-流動合計        6026
2   2330     2015_Q4              備供出售金融資產-流動淨額    14299361
3   2330     2015_Q4            持有至到期日金融資產-流動淨額     9166523
4   2330     2015_Q4               避險之衍生金融資產-流動        1739
..   ...         ...                        ...         ...
75  2330     2015_Q4                       權益總計  1222634479
76  2330     2015_Q4                    負債及權益總計  1657518298
77  2330     2015_Q4                    待註銷股本股數           0
78  2330     2015_Q4          預收股款(權益項下)之約當發行股數           0
79  2330     2015_Q4  母公司暨子公司所持有之母公司庫藏股股數(單位：股)           0

[80 rows x 4 columns]
profit&lose
     sID year_season                                  item      value
0   2330     2015_Q4                                銷貨收入淨額  843497368
1   2330     2015_Q4                                營業收入合計  843497368
2   2330     2015_Q4       

#  finance_report---Equalty

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

def equalty(sID,year,season):
    url = f"https://mops.twse.com.tw/server-java/t164sb01?step=1&CO_ID={sID}&SYEAR={year}&SSEASON={season}&REPORT_ID=C"
    res = requests.get(url)
    res.encoding = 'big5'
    soup = bs(res.text, "lxml")
    tables = soup.select('table')

    season = f"{year}_Q{season}"

    table = 4


    tbl = []
    column = []
    tr_list = tables[table].select("tr")
    for tr in tr_list:
        td_list = tr.select("td")
        th_list = tr.select("th")
        row = []
        for i,td in enumerate(td_list):
            row.append(td.text.strip().replace("\n",""))
        if len(row) > 0:
            tbl.append(row)

        for j,th in enumerate(th_list):
            column.append(th.text.strip().replace("\n",""))

    df = pd.DataFrame(tbl,columns = column)
    print(df)

    
sID = "2505"
year = "2018"
season = "4"
equalty(sID,year,season)

                       普通股股本       股本合計     資本公積   法定盈餘公積    特別盈餘公積  \
0            期初餘額  5,765,825  5,765,825  488,975  357,986   387,506   
1   追溯適用及追溯重編之影響數          0          0        0        0         0   
2         期初重編後餘額  5,765,825  5,765,825  488,975  357,986   387,506   
3        提列法定盈餘公積          0          0        0   14,409         0   
4         普通股現金股利          0          0        0        0         0   
5        特別盈餘公積迴轉          0          0        0        0  -387,506   
6        本期淨利（淨損）          0          0        0        0         0   
7        本期其他綜合損益          0          0        0        0         0   
8        本期綜合損益總額          0          0        0        0         0   
9            現金增資  1,200,000  1,200,000  138,708        0         0   
10        非控制權益增減          0          0        0        0         0   
11      權益增加(減少)數  1,200,000  1,200,000  138,708   14,409  -387,506   
12           期末餘額  6,965,825  6,965,825  627,683  372,395         0   

   未分